# Summarize News articles from main page of a news website
## Spacy Library
Initially considere using Flair but decided to switch to Spacy which is better in the sense that it is "industrial strength" and speed.

## Main Ideas and How the results can be used
* pick a news outlet
* filter news that mentions the target word or phrase  
* summarize article to get gist of news 

### Sift through news articles using python library *newspaper3k*  
https://newspaper.readthedocs.io/en/latest/

In [ ]:
#pip install newspaper3k

In [ ]:
import newspaper
from newspaper import Article
import re

In [ ]:
# build newspaper object 
wp_paper = newspaper.build('https://www.washingtonpost.com/', memoize_articles=False)
print(len(wp_paper.articles)) # number of articles on the front page

In [ ]:
# get urls of all the articles and store in a list
urls = []
for article in wp_paper.articles:
    urls.append(article.url)


In [ ]:
#!python3 -m pip install spacy 

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
### make a list of stopwords, import punctuation and add newline to it

stopwords = list(STOP_WORDS)
from string import punctuation
punctuation = punctuation+ '\n'

need to do below command for a better trained dictionary en_core_web_sm

In [ ]:
#!python3 -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load('en_core_web_sm')

for finding score of each word in a sentence, we use pytextrank


In [ ]:
#pip install pytextrank

In [ ]:
import pytextrank

In [ ]:
## add PyTextRank to the spaCy pipeline (do it once only otherwise it gives error)
nlp.add_pipe("textrank")


In [ ]:
#nlp.pipe_names # check if textrank got added

### Helper functions

In [ ]:
# function that returnss list contains all articles on front page that mention target word
def desired_articles(word):
    texts  = []
    for u in urls[:100]:
        article = Article(u)
        article.download()
        article.parse()
        if word in article.text:
            texts.append(article.text)
    return texts

In [ ]:
# define function for cleaning sentences before applying textrank 
#that will help later when calculating score of sentences.

def clean_text(t):
    return re.sub("[^a-zA-Z\.\?\!]", " ",t)

In [ ]:
# function that takes in set of sentences or a text block(t) and gives dictionay of score 
#of keyphrases
def dict_kphrase(t):
    dict_kp = dict() 
    doc = nlp(clean_text(t))
    for phrase in doc._.phrases:
        dict_kp[phrase.text]=phrase.rank
    return dict_kp

In [ ]:
# for spliting text into sentences we will use nltk
import nltk
#nltk.download('punkt') # one time execution

#function that returns list of lists (tokenized sentences)
def tokenize_sentences(t):
    list_sentences = []
    for sents in t:
        list_sentences.append(nltk.sent_tokenize(clean_text(sents)))
    return list_sentences

In [ ]:
# function that gives dictionary of score of each sentence using keyphrase scores. Takes in 
#list of sentences (sentences) and a dictionary (d) of scores of keyphrases of that text
def score(sentences,d): #changed t to sentences for clarity
    score_dict = dict()
    for sent in sentences:
        score_dict[sent] = 0
        for kphrase in d.keys():
            if kphrase in sent:
                score_dict[sent]+= d[kphrase]
    return score_dict
    

### Summary of articles:
The idea is to consider n highest ranked sentences as summary of the article.

In [ ]:
from heapq import nlargest

In [ ]:
# function to obtain summary of articles that mention the desired word.
def summary(word):
    t = desired_articles(word)
    list_sentences = tokenize_sentences(t) # list of lists
    summary = []
    for i in range(len(t)):
        d = dict_kphrase(t[i])
        score_dict = score(list_sentences[i],d)
        ThreeHighest = nlargest(3, score_dict, key = score_dict.get)
        summary.append(ThreeHighest)
    return(summary)

In [ ]:
summary("vaccine")